# mysql과 python 연동
1. 라이브러리 설치
2. sql 서버의 정보를 등록
3. sql 서버에 접속
4. sql 쿼리문을 이용하여 데이터를 CRUD 작업

In [3]:
# pymysql (라이브러리) 설치

!pip install pymysql

In [4]:
import pymysql

In [5]:
# sql 서버의 정보를 입력

_db = pymysql.connect(
    user = 'root',
    password = '1234',
    host = 'localhost',
    db = 'ubion7',
    port = 3306
)

In [6]:
# cursor 변수 생성
# sql 쿼리문을 실행 시키기 위한 필수 변수
# Dictcursor는 sql 쿼리문이 실행되고 나온 데이터를 딕셔너리 형태로 받는다.
cursor = _db.cursor(pymysql.cursors.DictCursor)

In [7]:
# """""" : 줄이 바뀌어도 이 사이는 다 str
sql = """
    select
    *
    from
    user_info
"""

cursor.execute(sql)

4

In [8]:
result = cursor.fetchall()

In [9]:
import pandas as pd

In [10]:
pd.DataFrame(result)

,user_id,user_pass,user_name,user_phone,user_gender
0,test,1234,test,01012345678,1
1,test2,1234,test2,01011111111,2
2,test3,1111,test3,01099999999,2
3,test4,1234,test4,01011111111,1


In [11]:
sql = """
    select
    *
    from
    example    
"""

cursor.execute(sql)         # execute로 sql의 실행 
result = cursor.fetchall()  # fetchall로 sql을 받아온다
df = pd.DataFrame(result)
df

,Name,Gender,Phone
0,A,male,010-1234-5678
1,B,female,010-1234-5679
2,C,male,010-1234-5680
3,D,female,010-1234-5681
4,E,male,010-1234-5682
5,F,female,010-1234-5683
6,G,male,010-1234-5684
7,H,female,010-1234-5685
8,I,male,010-1234-5686
9,J,female,010-1234-5687


In [12]:
sql = """
    insert into
    user_info
    values
    ("test5", "1234", "test5", "01011118888", 1)
"""

cursor.execute(sql)

# 여기까지 하면 test5 실제 데이터베이스에 적용되지 않는다.
# execute 한 다음, commit 해야 한다.

1

In [13]:
# commit
_db.commit()

# 여기까지 하고 MySQL에서 번개 누르거나, ctrl+enter 하면 변경 사항이 반영된다.

### sql 함수화
1. 첫 번째 함수
    - 매개변수 1개 (sql 쿼리문)
    - sql 쿼리문을 실행하고 결과를 데이터프레임으로 변경하여 리턴
2. 두 번째 함수
    - 매개변수 1개 (sql 쿼리문)
    - sql 쿼리문을 실행하고 데이터베이스를 변경
3. 세 번째 함수
    - 1번 함수와 2번 함수를 하나의 함수에서 실행이 될 수 있도록 생성

In [24]:
# 첫 번째 함수 (select 실행)
def sql_select(sql):
    
    # 쿼리문 실행
    cursor.execute(sql)  
    
    # 쿼리문에 대한 결과
    result = cursor.fetchall()

    df = pd.DataFrame(result)

    return df
    
    

In [25]:
sql = """
    select
    *
    from
    user_info    
"""

sql_select(sql)

,user_id,user_pass,user_name,user_phone,user_gender
0,test,1234,test,01012345678,1
1,test2,1234,test2,01011111111,2
2,test3,1111,test3,01099999999,2
3,test4,1234,test4,01011111111,1
4,test5,1234,test5,01011118888,1


In [30]:
# 두 번째 함수 (read, update, delete 모두 실행)
def _sql(sql, values) :
    
    cursor.execute(sql, values)
    _db.commit()
    

In [31]:
sql = """ 
    insert into
    user_info
    values
    (%s, %s, %s, %s, %s)

"""

values = ["test10", "12234", "test10", "01077777777", 1]

_sql(sql, values)

In [38]:
sql = """
    select
    *
    from
    user_info
"""

sql.find('select')
sql.replace

5

In [35]:
sql = """ 
    insert
    *
    from
    user_info
"""

sql.find('select')

-1

In [41]:
def sql_query(sql, values = []) :
    # if (sql.find('select') != -1) and (sql.find('select') < 10) :
    if (sql.replace('\n', '').strip().startswith('select')) :
        cursor.execute(sql, values)
        result = cursor.fetchall()
        result = pd.DataFrame(result)

    else :
        cursor.execute(sql, values)
        _db.commit()
        result = "Query OK"
    return result

In [42]:
sql = """
    select
    *
    from
    user_info
"""

sql_query(sql)

,user_id,user_pass,user_name,user_phone,user_gender
0,test,1234,test,01012345678,1
1,test10,12234,test10,01077777777,1
2,test2,1234,test2,01011111111,2
3,test3,1111,test3,01099999999,2
4,test4,1234,test4,01011111111,1
5,test5,1234,test5,01011118888,1
6,test6,1234,test6,01011111111,2


In [43]:
sql = """
    delete
    from
    user_info
    where
    user_id = %s
"""
    # values값을 여러개로 넣고 싶으면
    # user_id in (%s,.. 갯수만큼)

values = ['test10']

sql_query(sql, values)

'Query OK'

In [ ]:
import mod_sql